#dataset : Amazon fine food reviews

#Source : https://www.kaggle.com/laowingkin/amazon-fine-food-review-sentiment-analysis

In [ ]:
!pip3 install numpy

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from string import punctuation
from sklearn import svm
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
from nltk import ngrams
#from itertools import chain
#from wordcloud import WordCloud

In [3]:
# load dataset
df = pd.read_csv('/home/shiv/Dataset/Reviews.csv')

 y_dict = {1:0, 2:0, 4:1, 5:1}


df1 = pd.DataFrame({'score':[1,2,3,4,5], 'uid': [12,23,45,56,43]})
temp_score=  df1['score'].map(y_dict)
df1.score = temp_score
print(df1)


In [4]:
# removing nuetral reviews, with value 3 
#df = df[df['Score'] != 3]

#mapping score with poistive and negative

def partition(x):
    if x < 3:
        return '1'
    return '0'

temp_score = df.Score
ts = temp_score.map(partition)
df.Score = ts

In [5]:
X = df['Text']
y = df.Score
df.columns

Index(['Id', 'ProductId', 'UserId', 'ProfileName', 'HelpfulnessNumerator',
       'HelpfulnessDenominator', 'Score', 'Time', 'Summary', 'Text'],
      dtype='object')

In [6]:
X.head()

0    I have bought several of the Vitality canned d...
1    Product arrived labeled as Jumbo Salted Peanut...
2    This is a confection that has been around a fe...
3    If you are looking for the secret ingredient i...
4    Great taffy at a great price.  There was a wid...
Name: Text, dtype: object

# Logistic regression model on word count


In [7]:
c = CountVectorizer(stop_words = 'english')

def text_fit(X, y, model,clf_model,coef_show=1):
    
    X_c = model.fit_transform(X)
    #print('# features: {}'.format(X_c.shape[1]))
    #split into train and test 
    X_train, X_test, y_train, y_test = train_test_split(X_c, y, random_state=0)
   # print('# train records: {}'.format(X_train.shape[0]))
    #print('# test records: {}'.format(X_test.shape[0]))
    clf = clf_model.fit(X_train, y_train)
    acc = clf.score(X_test, y_test)
    print ('Model Accuracy: {}'.format(acc))
    
    if coef_show == 1: 
        w = model.get_feature_names()
        coef = clf.coef_.tolist()[0]
        coeff_df = pd.DataFrame({'Word' : w, 'Coefficient' : coef})
        coeff_df = coeff_df.sort_values(['Coefficient', 'Word'], ascending=[0, 1])
        print('')
        print('-Top 20 positive-')
        print(coeff_df.head(20).to_string(index=False))
        print('')
        print('-Top 20 negative-')        
        print(coeff_df.tail(20).to_string(index=False))
    
    
text_fit(X, y, c, LogisticRegression())

/home/shiv/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Model Accuracy: 0.9234347073476223

-Top 20 positive-
         Word  Coefficient
  undrinkable     2.936746
       ripoff     2.815729
    deceptive     2.696162
  disapointed     2.678524
        worst     2.407214
        lousy     2.405120
        blech     2.368444
         lame     2.353332
   unbearable     2.346018
        ruins     2.244851
      weakest     2.241170
  embarrassed     2.218491
      vomited     2.202253
         vile     2.187482
        schar     2.179580
   returnable     2.165382
      defeats     2.106338
 unacceptable     2.105772
          ick     2.081416
     vinegary     2.071368

-Top 20 negative-
         Word  Coefficient
     terrific    -1.668224
 unmistakable    -1.690465
      easiest    -1.716821
        steal    -1.730062
      gobbles    -1.746297
    amazingly    -1.750267
       brings    -1.769412
      trainer    -1.797404
      drained    -1.847494
      worries    -1.849322
 conventional    -1.959105
     drawback    -1.966748
       de

# Logistic regression model on TFIDF

In [8]:
tfidf = TfidfVectorizer(stop_words = 'english')
text_fit(X, y, tfidf, LogisticRegression())

/home/shiv/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Model Accuracy: 0.9224284729161096

-Top 20 positive-
           Word  Coefficient
          worst    10.376925
       terrible     8.594575
          awful     8.197440
  disappointing     7.696216
       horrible     7.340680
   disappointed     6.832412
 disappointment     6.764436
          threw     6.514545
     disgusting     6.297724
      tasteless     6.268056
           yuck     6.084551
    undrinkable     6.029501
          waste     5.801447
           poor     5.662441
         return     5.592851
          gross     5.563182
          nasty     5.479702
  unfortunately     5.315167
        useless     5.251654
          worse     5.023765

-Top 20 negative-
       Word  Coefficient
       easy    -5.522381
       beat    -5.544556
      yummy    -5.821393
  fantastic    -5.851143
   favorite    -6.183643
    awesome    -6.245762
 pleasantly    -6.317389
    amazing    -6.417974
       nice    -6.509099
       good    -6.584429
  wonderful    -6.808270
     hooked    -6.

# Logistic regression model on TFIDF + ngram¶


In [ ]:
tfidf_n = TfidfVectorizer(ngram_range=(1,2),stop_words = 'english')
text_fit(X, y, tfidf_n, LogisticRegression())